In [14]:
with open('data/cve_desc.pickle', 'rb') as f:
    cve_desc = pk.load(f)


In [16]:
#only consider applications and lower the case
cve_desc = {cve_id:{'description' : cve_info['description'].lower(),
                   'cpes' : [cpe.lower() for cpe in cve_info['cpes'] if cpe[8]=='a']} for (cve_id,cve_info) in cve_desc.items()}

cve_desc = {cve_id:cve_desc[cve_id] for cve_id in cve_desc if cve_desc[cve_id]['cpes']}


In [19]:
cve_desc

{'CVE-2005-3331': {'description': 'viewpatch in mgdiff 1.0 allows local users to overwrite arbitrary files via a symlink attack on temporary files.',
  'cpes': ['cpe:2.3:a:rogers_software_source:mgdiff_patch_viewer:1.0:*:*:*:*:*:*:*']},
 'CVE-2005-3330': {'description': 'the _httpsrequest function in snoopy 1.2, as used in products such as (1) magpierss, (2) wordpress, (3) ampache, and (4) jinzora, allows remote attackers to execute arbitrary commands via shell metacharacters in an https url to an ssl protected web page, which is not properly handled by the fetch function.',
  'cpes': ['cpe:2.3:a:snoopy:snoopy:1.2:*:*:*:*:*:*:*']},
 'CVE-2005-3333': {'description': 'sql injection vulnerability in ebaseweb 3.0 allows remote attackers to execute arbitrary sql commands via unknown attack vectors.',
  'cpes': ['cpe:2.3:a:ebase:ebaseweb:3.0:*:*:*:*:*:*:*']},
 'CVE-2005-3332': {'description': 'php remote file include vulnerability in admin/define.inc.php in belchior foundry vcard 2.9 allows 

In [333]:
descriptions = {i:cve_desc[i]['description'] for i in cve_desc}

In [328]:
#find .js in cpe and description

import re
cve_wo_js = {} #without .js
cve_w_js = set() #with .js
js_dict = {}
for cve_id in cve_desc:
    for cpe in cve_desc[cve_id]['cpes']:
        js_name_list = re.findall('[\w\-\.]*\.js(?!p|on)', cpe)
        if js_name_list:
            for js_name in js_name_list:
                cve_w_js.add(cve_id)
                js_name = js_name[:-3]
                if js_name not in js_dict:
                    js_dict[js_name] = []
                js_dict[js_name].append(cve_id)
        else:
            cve_wo_js[cve_id] = cve_desc[cve_id]
            
    desc = cve_desc[cve_id]['description']
    js_name_list = re.findall('[\w\-\.]*\.js(?!p|on)', desc)
    if js_name_list:
        for js_name in js_name_list:
            cve_w_js.add(cve_id)
            js_name = js_name[:-3]
            if js_name not in js_dict:
                js_dict[js_name] = []
            js_dict[js_name].append(cve_id)
    else:
        cve_wo_js[cve_id] = cve_desc[cve_id]

In [330]:
len(cve_desc)

93833

In [331]:
len(cve_wo_js)

93703

In [334]:

import re
word = 'jquery'
a = [k for (k,v) in descriptions.items() if re.search('(?<!\w)'+word+'(?!\.js|\w)',v)] #search word not followed by .js
b = [k for (k,v) in descriptions.items() if re.search('(?<!\w)'+word+'\.js(?!on|p)',v)] #search word followed by .js

In [29]:
print(len(a))
print(len(b))

32
1


In [335]:
# i=-1
i+=1
print(a[i])
print(cve_desc[a[i]])

CVE-2011-4969
{'description': 'cross-site scripting (xss) vulnerability in jquery before 1.6.3, when using location.hash to select elements, allows remote attackers to inject arbitrary web script or html via a crafted tag.', 'cpes': ['cpe:2.3:a:jquery:jquery:1.6:*:*:*:*:*:*:*', 'cpe:2.3:a:jquery:jquery:1.6.1:*:*:*:*:*:*:*', 'cpe:2.3:a:jquery:jquery:*:*:*:*:*:*:*:*']}


In [336]:
#build the corpus with vendor name
import copy
cve_desc_ori = copy.deepcopy(cve_desc)
for cve_id in cve_desc_green:
    cpes = []
    for cpe in cve_desc[cve_id]['cpes']:
        if cpe[8]=='a':
            cpe_ = cpe[10:]
            p1 = cpe_.find(':')
            p2 = cpe_[p1+1:].find(':')
            new_cpe = cpe_[:p1] + ' ' + cpe_[p1+1:p1+p2+1]
            if new_cpe not in cpes:
                cpes.append(new_cpe)
    
    cve_desc_ori[cve_id]['cpes'] = cpes

In [337]:
#build the corpus without vendor name
import copy
cve_desc_ori = copy.deepcopy(cve_desc)
for cve_id in cve_desc_green:
    cpes = []
    for cpe in cve_desc[cve_id]['cpes']:
        if cpe[8]=='a':
            cpe_ = cpe[10:]
            p1 = cpe_.find(':')
            p2 = cpe_[p1+1:].find(':')
            new_cpe = cpe_[p1+1:p1+p2+1]
            if new_cpe not in cpes:
                cpes.append(new_cpe)
    
    cve_desc_ori[cve_id]['cpes'] = cpes

In [338]:
cve_desc_ori

{'CVE-2005-3331': {'description': 'viewpatch in mgdiff 1.0 allows local users to overwrite arbitrary files via a symlink attack on temporary files.',
  'cpes': ['mgdiff_patch_viewer']},
 'CVE-2005-3330': {'description': 'the _httpsrequest function in snoopy 1.2, as used in products such as (1) magpierss, (2) wordpress, (3) ampache, and (4) jinzora, allows remote attackers to execute arbitrary commands via shell metacharacters in an https url to an ssl protected web page, which is not properly handled by the fetch function.',
  'cpes': ['snoopy']},
 'CVE-2005-3333': {'description': 'sql injection vulnerability in ebaseweb 3.0 allows remote attackers to execute arbitrary sql commands via unknown attack vectors.',
  'cpes': ['ebaseweb']},
 'CVE-2005-3332': {'description': 'php remote file include vulnerability in admin/define.inc.php in belchior foundry vcard 2.9 allows remote attackers to execute arbitrary php code via the match parameter.',
  'cpes': ['vcard']},
 'CVE-2005-3335': {'desc

In [339]:
for cve_id in cve_desc_ori:
    for cpe in cve_desc_ori[cve_id]['cpes']:
        if re.search('[0-9][^\d{1}][0-9]',cpe):
            print(cve_id,cpe)

CVE-2012-2023 illustrator_cs5.5
CVE-2012-2026 illustrator_cs5.5
CVE-2012-2024 illustrator_cs5.5
CVE-2012-2025 illustrator_cs5.5
CVE-2009-0761 1.0
CVE-2009-0761 1.0.1
CVE-2009-0761 1.0.2
CVE-2009-0761 1.0.3
CVE-2009-0761 1.0.4
CVE-2009-0761 1.0.5
CVE-2001-0955 x11r6
CVE-2012-2028 photoshop_cs5.5
CVE-2012-2027 photoshop_cs5.5
CVE-2014-0176 cloudforms_3.0_management_engine
CVE-2008-6761 flexcustomer0.0.6
CVE-2004-0419 x11r6
CVE-2012-3533 3.1.0.5
CVE-2009-1381 squirrelmail1.4.19-1
CVE-2014-4624 6.0
CVE-2014-4624 6.0.402
CVE-2014-4624 7.0
CVE-2014-4624 7.0.2-43
CVE-2010-5214 fotobook_editor_5.0
CVE-2006-0970 1-2-all
CVE-2013-5466 db2_purescale_feature_9.8
CVE-2017-5703 core_i7-8550u
CVE-2017-5703 core_i7-8559u
CVE-2017-5703 core_i7-8650u
CVE-2017-5703 core_i7-8700
CVE-2017-5703 core_i7-8700b
CVE-2017-5703 core_i7-8700k
CVE-2017-5703 core_i7-8700t
CVE-2017-5703 core_i7-8705g
CVE-2017-5703 core_i7-8706g
CVE-2017-5703 core_i7-8709g
CVE-2017-5703 core_i7-8750h
CVE-2017-5703 core_i7-8809g
CVE-20

CVE-2011-1522 doctrine1.2.1
CVE-2011-1522 doctrine1.2.2
CVE-2011-1522 doctrine1.2.3
CVE-2009-1943 softremote1.4
CVE-2012-0778 flash_cs5.5
CVE-2018-3700 usb_3.0_extensible_host_controller_driver
CVE-2018-17236 mp4v2
CVE-2012-3845 lan_messenger1.2.28
CVE-2017-14921 tine_2.0
CVE-2017-14922 tine_2.0
CVE-2017-14923 tine_2.0
CVE-2009-1350 netidentity_client1.2.3
CVE-1999-0241 x11r6
CVE-2014-0180 cloudforms_3.0_management_engine
CVE-2000-0453 x11r6
CVE-2014-6682 w88235ff7bdc2fb574f1789750ea99ed6
CVE-2007-3520 1-2-3_music_store
CVE-2005-4417 blue_usb-130-250_software
CVE-2017-1000164 tine_2.0
CVE-2011-5012 reflection_2008r1
CVE-2011-5012 reflection_2008r2
CVE-2011-5012 reflection_2011r1
CVE-2007-1351 x11r6
CVE-2015-0713 telepresence_ip_vcr_1.0_converter
CVE-2015-0713 telepresence_ip_vcr_2.4
CVE-2015-0713 telepresence_ip_vcr_3.0
CVE-2007-5951 0.2
CVE-2018-5779 st14.2
CVE-2005-2020 3c15100d
CVE-2005-3855 1-2-3_music_store
CVE-2005-0605 x11r6
CVE-2008-3952 2.0
CVE-2007-2630 1-2-all_broadcast_emai

In [340]:
def string_form(s):
    s = re.sub('[_\-/]', ' ', s) #substitute '_', '/', '-' with space
    s = re.sub('\.(?![\d])|(?<![\d])\.', ' ', s) #substitute dots that are not between digits with space
    s = re.sub(r'[\\]', '', s)
    return s

In [341]:
#cve after string transformation
import copy
import re
cve_desc_green = copy.deepcopy(cve_desc)
for cve_id in cve_desc_green:
    cpes = []
    for cpe in cve_desc_green[cve_id]['cpes']:
        if cpe[8]=='a':
            cpe_ = cpe[10:]
            p1 = cpe_.find(':')
            p2 = cpe_[p1+1:].find(':')
            new_cpe = string_form( cpe_[p1+1:p1+p2+1])
#             new_cpe = re.sub('[_.\-/]', ' ', cpe_[:p1] + ' ' + cpe_[p1+1:p1+p2+1])
#             new_cpe = re.sub(r'[\\]', '', new_cpe)
#             new_cpe = (cpe_[:p1] + ' ' + cpe_[p1+1:p1+p2+1]).replace('_',' ').replace('.',' ').replace('-',' ')
            if new_cpe not in cpes:
                cpes.append(new_cpe)
    cve_desc_green[cve_id]['cpes'] = cpes

In [342]:
cve_desc_green

{'CVE-2005-3331': {'description': 'viewpatch in mgdiff 1.0 allows local users to overwrite arbitrary files via a symlink attack on temporary files.',
  'cpes': ['mgdiff patch viewer']},
 'CVE-2005-3330': {'description': 'the _httpsrequest function in snoopy 1.2, as used in products such as (1) magpierss, (2) wordpress, (3) ampache, and (4) jinzora, allows remote attackers to execute arbitrary commands via shell metacharacters in an https url to an ssl protected web page, which is not properly handled by the fetch function.',
  'cpes': ['snoopy']},
 'CVE-2005-3333': {'description': 'sql injection vulnerability in ebaseweb 3.0 allows remote attackers to execute arbitrary sql commands via unknown attack vectors.',
  'cpes': ['ebaseweb']},
 'CVE-2005-3332': {'description': 'php remote file include vulnerability in admin/define.inc.php in belchior foundry vcard 2.9 allows remote attackers to execute arbitrary php code via the match parameter.',
  'cpes': ['vcard']},
 'CVE-2005-3335': {'desc

In [343]:
#dict from product name to cve id
cpes_2_cve = {}
for cve_id in cve_desc_green:
    for cpe in cve_desc_green[cve_id]['cpes']:
        if cpe not in cpes_2_cve:
            cpes_2_cve[cpe] = []
        cpes_2_cve[cpe].append(cve_id)

In [344]:
#import npm data
import pickle as pk
with open('data/npmPackageNames_2019-05-09_16:11:48.pickle','rb') as f:
    npm = pk.load(f)
npm = [string_form(i) for i in npm]

In [345]:
npm

['0',
 '0 ',
 '0    ',
 '0 1 project',
 '0 100',
 '0 24',
 '0 60',
 '0 9',
 '0   0',
 '0 ',
 '0.0',
 '0.0.1',
 '0.0.168',
 '0.0.250',
 '0.1.0',
 '0.1 unity settings daemon',
 '0.1f',
 '0.2.18',
 '0 css',
 '0 js',
 '0 workspace',
 '00',
 '00 components',
 '00 test',
 '00 demo',
 '000 webpack',
 '0003 lion lib',
 '001',
 '001 nodelist',
 '001 npm',
 '001 skt',
 '001 test',
 '001senge',
 '002',
 '002 bao',
 '002 globals',
 '002 npm',
 '002twoweek',
 '003',
 '003 3',
 '003 bingpic',
 '003 npm',
 '003threeweek',
 '004 bingpic',
 '004 module',
 '004 week',
 '005',
 '005 gxp',
 '005 http',
 '005 http antao',
 '005 http jin',
 '005 http open',
 '005 https cxb',
 '005http',
 '007',
 '008 mysql',
 '008 somepackage',
 '009',
 '01',
 '01 03',
 '01 bibliotheque',
 '01 calc',
 '01 cute',
 '01 dj',
 '01 hhtclac',
 '01 numacert',
 '01 plugin',
 '01 practicemyself',
 '01 simple',
 '01 szhm wb',
 '01 szhmqd25calc',
 '01 szhmqd27cale',
 '01 upload',
 '01.22pagination',
 '01 test cnpm',
 '010 static http 

# tf-idf for matching

In [4]:
import nltk
def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z0-9]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [49]:
#merge cpes and npm
all_cpes = [cpe for cpe in cpes_2_cve]
text = all_cpes + npm

In [51]:
def tokenized_vocab(text_list):
    totalvocab_tokenized = []
    for i in text_list:

        allwords_tokenized = tokenize_only(i)
        totalvocab_tokenized.extend(allwords_tokenized)
    return totalvocab_tokenized
totalvocab = set(tokenized_vocab(text))

In [52]:
with open('totalvocab.pkl','wb') as f:
    pk.dump(totalvocab, f)

#     totalvocab = pickle.load(f,'rb'))

In [53]:
len(totalvocab)

329075

In [54]:
# # tf-idf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(#min_df =10**-3 ,
                                   analyzer = 'word', max_features=len(set(totalvocab)), 
#                                    stop_words=my_stop_words, 
                                   tokenizer=tokenize_only, 
#                                     ngram_range=(1,3)
                                    )



In [55]:
tfidf_matrix = tfidf_vectorizer.fit_transform(text)
all_cpes_vec = tfidf_matrix[:len(all_cpes)]
npm_vec = tfidf_matrix[len(all_cpes):]
grade = all_cpes_vec.dot(npm_vec.transpose())

most_arg = np.argmax(grade, axis=1)

most_grade = np.max(grade, axis=1)
all_match = list(zip(np.array(all_cpes),np.array(npm)[most_arg].reshape((len(all_cpes))), most_grade.toarray().reshape(-1)))

In [56]:
with open( "tfidf_matrix.pkl", "wb" ) as f:
    pk.dump(tfidf_matrix, f)
    # tfidf_matrix = pickle.load(f)
with open( "tfidf_vectorizer.pkl", "wb" ) as f:
    pk.dump(tfidf_vectorizer, f)
    # tfidf_vectorizer = pickle.load(f)

In [57]:
#vectorization
all_cpes_vec = tfidf_matrix[:len(all_cpes)]
npm_vec = tfidf_matrix[len(all_cpes):]

In [58]:
#compute grade and find best matches
grade = all_cpes_vec.dot(npm_vec.transpose())

most_arg = np.argmax(grade, axis=1)

most_grade = np.max(grade, axis=1)

In [59]:
all_match = list(zip(np.array(all_cpes),np.array(npm)[most_arg].reshape((len(all_cpes))), most_grade.toarray().reshape(-1)))

In [161]:
all_match

[('mgdiff patch viewer', 'patch', 0.46975437103935863),
 ('snoopy', 'snoopy', 1.0),
 ('ebaseweb', '0', 0.0),
 ('vcard', 'vcard', 1.0),
 ('mantis', 'mantis', 1.0),
 ('flyspray', '0', 0.0),
 ('python', 'python', 1.0),
 ('postgresql', 'postgresql', 1.0),
 ('openssh', 'openssh rsa dsa parse', 0.6189807681860662),
 ('unified threat management software', 'threat', 0.5886744885675435),
 ('tomee', '0', 0.0),
 ('simplesamlphp', '0', 0.0),
 ('saml2', 'saml2', 1.0),
 ('gstreamer', 'gstreamer', 1.0),
 ('libav', '0', 0.0),
 ('jscript', 'jscript', 1.0),
 ('vbscript', 'vbscript to typescript', 0.8258526671011653),
 ('internet explorer', 'internet explorer version', 0.862816514766701),
 ('firefox', 'firefox', 1.0),
 ('backupninja', '0', 0.0),
 ('pillow', 'pillow', 1.0),
 ('remote device access virtual customer access system',
  'remote access',
  0.7083000871554723),
 ('connections business directory plugin', 'connections', 0.6321427907792957),
 ('samba', 'samba', 1.0),
 ('infosphere information serve

In [169]:
#frequencies of all ranges of grades
match_ranges = {round(10*k)/10:len([i for i in all_match if k+0.1>i[2]>=k]) for k in np.arange(1,-0.1,-0.1)}
print(match_ranges)
print(np.array(list(match_ranges.values()))/sum(match_ranges.values()))

{1.0: 3556, 0.9: 642, 0.8: 2311, 0.7: 4682, 0.6: 4445, 0.5: 4352, 0.4: 2337, 0.3: 341, 0.2: 16, 0.1: 0, 0.0: 0}
[0.1567763  0.02830438 0.10188696 0.20641919 0.19597037 0.19187021
 0.10303324 0.01503395 0.00070541 0.         0.        ]


In [216]:
has_list = []
for cve_id in cve_desc_green:
    cpes = cve_desc_green[cve_id]['cpes']
    alltext = ' '.join(cpes) +cve_desc_green[cve_id]['description']
    for cpe in cpes:
        if re.search('jquery', alltext):
#         if cpe=='fuse':
            has_list.append(cve_id)


In [327]:
# i=-1
i+=1
print(i)
cve_desc[has_list[i]]

1


{'description': 'mtappjquery 1.8.1 and earlier allows remote php code execution via unspecified vectors.',
 'cpes': ['cpe:2.3:a:bit-part:mtappjquery:*:*:*:*:*:movabletype:*:*']}

In [278]:
len(has_list)

46

In [210]:
i=-1
i+=1
cve_desc[a[i]]

{'description': 'cross-site scripting (xss) vulnerability in the datatables plugin 1.10.8 and earlier for jquery allows remote attackers to inject arbitrary web script or html via the scripts parameter to media/unit_testing/templates/6776.php.',
 'cpes': ['cpe:2.3:a:sprymedia:datatables:*:*:*:*:*:jquery:*:*']}

In [162]:
match_100 = [i for i in all_match if 1<=i[2]]
len(match_100)

3556

In [163]:
#perfect match
match_100

[('snoopy', 'snoopy', 1.0),
 ('vcard', 'vcard', 1.0),
 ('mantis', 'mantis', 1.0),
 ('python', 'python', 1.0),
 ('postgresql', 'postgresql', 1.0),
 ('saml2', 'saml2', 1.0),
 ('gstreamer', 'gstreamer', 1.0),
 ('jscript', 'jscript', 1.0),
 ('firefox', 'firefox', 1.0),
 ('pillow', 'pillow', 1.0),
 ('samba', 'samba', 1.0),
 ('vlc media player', 'vlc media player', 1.0),
 ('openrefine', 'openrefine', 1.0),
 ('popcorn', 'popcorn', 1.0),
 ('freetype', 'freetype', 1.0),
 ('safari', 'safari', 1.0),
 ('cups', 'cups', 1.0),
 ('ie', 'ie', 1.0),
 ('opencv', 'opencv', 1.0),
 ('qpdf', 'qpdf', 1.0),
 ('office', 'office', 1.0),
 ('server', 'Server', 1.0),
 ('visio', 'visio', 1.0),
 ('works', 'works', 1.0),
 ('ftp', 'ftp', 1.0),
 ('sendmail', 'sendmail', 1.0),
 ('mesa', 'mesa', 1.0),
 ('mutt', 'mutt', 1.0),
 ('ssh', 'ssh', 1.0),
 ('global', 'global', 1.0),
 ('binutils', 'binutils', 1.0),
 ('word', 'word', 1.0),
 ('excel', 'excel', 1.0),
 ('zero cms', 'cms zero', 1.0),
 ('endless', 'endless', 1.0),
 ('unz

In [63]:
match_90 = [i for i in all_match if 1>i[2]>=0.9]
len(match_90)

642

In [64]:
match_90

[('wireshark', 'node wireshark', 0.9401619617812456),
 ('jrun', 'jrun js', 0.9424776543929972),
 ('report viewer', 'report viewer', 0.9999999999999999),
 ('poppler', 'poppler simple', 0.9054465550396035),
 ('typo3', 'generator typo3', 0.9185011275897756),
 ('exchange server', 'exchange test server', 0.9018773456407552),
 ('teamspeak3', 'teamspeak3 client', 0.9211198351487024),
 ('libshout', 'libshout js', 0.9424776543929972),
 ('application server', 'react application server', 0.934701271966239),
 ('hylafax', 'hylafax client', 0.9211198351487024),
 ('skeleton theme', 'react theme skeleton', 0.9410919140464948),
 ('virtuemart', 'virtuemart api', 0.9233583537857097),
 ('business manager', 'business manager', 0.9999999999999998),
 ('plesk', '@plesk plesk ext sdk', 0.9175482888617823),
 ('apple tv', 'apple tv', 0.9999999999999998),
 ('libpng', 'node libpng', 0.9424344106729406),
 ('cloud backup', 'cloud backup server', 0.9007764213332878),
 ('weblogic server', 'weblogic', 0.905029383988992

In [67]:
#import ground truth data
with open('data/npm_package_name_with_matching_cveid.pickle','rb') as f:
    npm_match_ground_truth = pk.load(f)

In [68]:
npm_match_ground_truth

[('datatables', 'CVE-2015-6584'),
 ('grunt-webdriver-qunit', 'CVE-2016-10606'),
 ('panels.js', 'CVE-2015-8861'),
 ('sfml', 'CVE-2016-10654'),
 ('ember', 'CVE-2014-0014'),
 ('looppake', 'CVE-2017-16169'),
 ('kibana', 'CVE-2017-8452'),
 ('backbone', 'CVE-2016-10537'),
 ('ldapauth-fork', 'CVE-2015-7294'),
 ('node', 'CVE-2016-5325'),
 ('windows-selenium-chromedriver', 'CVE-2016-10687'),
 ('cobalt-cli', 'CVE-2016-10597'),
 ('zeroclipboard', 'CVE-2014-1869'),
 ('electron', 'CVE-2017-12581'),
 ('keycloak-auth-utils', 'CVE-2017-7474'),
 ('pandora-doomsday', 'CVE-2017-16127'),
 ('cue-sdk-node', 'CVE-2016-10590'),
 ('gaoxuyan', 'CVE-2017-16153'),
 ('node', 'CVE-2017-14919'),
 ('bkjs-wand', 'CVE-2016-10571'),
 ('mediaelement', 'CVE-2016-4567'),
 ('string', 'CVE-2017-16116'),
 ('dylmomo', 'CVE-2017-16163'),
 ('assembly', 'CVE-2015-8861'),
 ('galenframework-cli', 'CVE-2016-10560'),
 ('mapbox.js', 'CVE-2017-1000043'),
 ('nodemailer.js', 'CVE-2017-16072'),
 ('lz4', 'CVE-2014-4611'),
 ('kibana', 'CVE-

In [185]:
a = [i[1] for i in npm_match_ground_truth]
b = [string_form(i[0]) for i in npm_match_ground_truth]

In [146]:
#npm for ground truth
b_unique = list(set(b))

In [79]:
no_index = []
for i in range(len(b)):
    if b[i] not in npm:
        no_index.append(i)
print(no_index)

[53, 54, 94, 132, 212, 238, 277, 290, 332, 357, 387, 490, 531, 541, 547, 548, 614, 678, 687, 711, 792]


In [82]:
# cpe for ground truth
cpe_list_ = []
for i in range(len(npm_match_ground_truth)):
    cve_id = npm_match_ground_truth[i][1]
    if cve_id in cve_desc_green:
        cpe_list_.extend(cve_desc_green[cve_id]['cpes'])
cpe_list_ = list(set(cpe_list_))

In [147]:
all_cpes_ = [cpe for cpe in cpe_list_]
text_ = all_cpes_ + b_unique

In [126]:

tfidf_matrix_ = tfidf_vectorizer.transform(text_)
all_cpes_vec_ = tfidf_matrix_[:len(all_cpes_)]
npm_vec_ = tfidf_matrix_[len(all_cpes_):]
grade_ = all_cpes_vec_.dot(npm_vec_.transpose())

most_arg_ = np.argmax(grade_, axis=0)

most_grade_ = np.max(grade_, axis=0)


In [127]:
npm_vec_.shape[0]

641

In [148]:
all_match_ = list(zip(np.array(all_cpes_)[most_arg_].reshape((npm_vec_.shape[0])),np.array(b_unique), most_grade_.toarray().reshape(-1)))

In [129]:
len(all_match_)

641

In [131]:
match_90_ = [i for i in all_match_ if 1>i[2]>=0.9]
len(match_90_)

81

In [132]:
match_80_ = [i for i in all_match_ if 0.9>i[2]>=0.8]
len(match_80_)

10

In [149]:
match_ranges = {round(10*k)/10:len([i for i in all_match_ if k+0.1>i[2]>=k]) for k in np.arange(1,-0.1,-0.1)}
print(match_ranges)
print(np.array(list(match_ranges.values()))/sum(match_ranges.values()))

{1.0: 456, 0.9: 81, 0.8: 10, 0.7: 10, 0.6: 10, 0.5: 14, 0.4: 10, 0.3: 10, 0.2: 6, 0.1: 0, 0.0: 0}
[0.75123558 0.13344316 0.01647446 0.01647446 0.01647446 0.02306425
 0.01647446 0.01647446 0.00988468 0.         0.        ]


In [164]:
[i for i in all_match_ if 0.8>i[2]>=0.7]

[('adm zip', 'adm zip mit', 0.7876956047126276),
 ('flowplayer html5', 'flowplayer', 0.7940680475299016),
 ('adm zip', 'adm zip with enc', 0.7096629221767243),
 ('serve', 'serve index', 0.727090996370414),
 ('query mysql', 'mysql', 0.7205384828635941),
 ('ember js', 'ember', 0.7844933642504155),
 ('adm zip', 'adm zip iconv', 0.7895573702298826),
 ('auth0', 'auth0 lock', 0.7148464281458531),
 ('node js', 'node', 0.7072518992211563),
 ('ejs', 'ejs co', 0.7340395644158756)]

In [354]:
#import mvn data
import json

with open('data/parsed_mvn_cent_idx_solr_06032019.json') as f:
    mvn = json.load(f)

In [356]:
mvn_all = {}
for i in range(len(mvn)):
    artifact_u = string_form(mvn[i].get('artifact_u',''))
    if artifact_u not in mvn_all:
        mvn_all[artifact_u] = (i, string_form(mvn[i].get('group_u','')))
mvn_all


{'yom': (0, 'yom'),
 'ymsg test': (2, 'ymsg'),
 'ymsg support': (3, 'ymsg'),
 'ymsg network': (4, 'ymsg'),
 'yan': (6, 'yan'),
 'jfunutil': (13, 'yan'),
 'xxl': (19, 'xxl'),
 'xtiff jai': (20, 'xtiff jai'),
 'xstream': (21, 'xstream'),
 'xsdlib': (48, 'xsdlib'),
 'xsddoc': (49, 'xsddoc'),
 'maven xsddoc plugin': (54, 'xsddoc'),
 'xpp3 xpath': (58, 'xpp3'),
 'xpp3 min': (62, 'xpp3'),
 'xpp3': (67, 'xpp3'),
 'xom': (81, 'xom'),
 'xmlwriter': (93, 'xmlwriter'),
 'xmlunit': (111, 'xmlunit'),
 'xmlrpc helma': (118, 'xmlrpc helma'),
 'xmlrpc server': (119, 'xmlrpc'),
 'xmlrpc common': (124, 'xmlrpc'),
 'xmlrpc client': (128, 'xmlrpc'),
 'xmlrpc': (132, 'xmlrpc'),
 'xmlpull': (150, 'xmlpull'),
 'xfc': (154, 'xmlmind'),
 'aptconvert': (155, 'xmlmind'),
 'xmlenc': (156, 'xmlenc'),
 'xmldb xupdate': (159, 'xmldb'),
 'xmldb common': (160, 'xmldb'),
 'xmldb api sdk': (161, 'xmldb'),
 'xmldb api': (162, 'xmldb'),
 'xmlpublic': (163, 'xmlbeans'),
 'xmlbeans xmlpublic': (167, 'xmlbeans'),
 'xmlbeans'